# Librerías Necesarias

In [3]:
#!pip install shap
#!pip install umap
!#pip install pyfpgrowth

"#pip" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [4]:
import gc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import pyfpgrowth
import seaborn as sns
import shap
import umap
import warnings

In [5]:
import sys
sys.setrecursionlimit(16385)

In [6]:
#Libreamos espacio en la GPU
gc.collect()

0

In [7]:
#Para visualizar todas las columnas
pd.set_option('display.max_columns', None)

In [8]:
#Eliminar las advertencias 
warnings.filterwarnings("ignore")

# Lectura de Datos 
Cargamos los datasets resultantes

In [9]:
base_path = "./ConjuntosDatos"
endutih_path = f"{base_path}/ENDUTIH2020/conjuntos_de_datos"

In [10]:
hogares_agrupados = pd.read_csv(f"{endutih_path}/viv_hog_agrupados.csv")
hogares_agrupados

,P1_1,P1_2,P1_3,P1_4,P1_5_1,P1_5_2,P1_5_3,P2_1,P2_2,P2_3,FAC_VIV,P4_1_1,P4_1_2,P4_1_3,P4_1_4,P4_1_5,P4_1_6,P4_1_6_1,P4_1_6_2,P4_2_1,P4_2_2,P4_2_3,P4_3,P4_4,P4_5,P4_6_1,P4_6_2,P4_6_3,P4_6_4,P4_6_5,P4_6_6,P4_7_1,P4_7_2,P4_7_3,P4_7_4,P4_8,P5_1,P5_2_1,P5_2_2,P5_3_1,P5_3_2,P5_4,P5_5,P5_6_1,P5_6_2,P5_6_3,P5_6_4,P5_6_5,P5_7_1,P5_7_2,P5_8_2,P5_9_2,P5_10_2,P5_7_3,P5_8_3,P5_7_4,P5_8_4,P5_9_4,P5_7_5,P5_8_5,P5_7_6,P5_8_6,P5_7_7,P5_8_7,P5_7_8,P5_8_8,P5_11_1,P5_11_2,P5_11_3,FAC_HOG,DOMINIO,TLOC,ESTRATO,Grupo,UPM,UPM_DIS,EST_DIS,ENT,HOGAR,VIV_SEL
0,2,1,1,1,1,2,2,2,1,1.0,572,2,1,1,2,2,1,2.0,2.0,2,2,2,1.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2,0.0,0.0,0.0,0.0,1.0,2,2,2,2,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,3.0,3.0,3.0,575,U,1,1,11,206264,570,17,2,1,1
1,2,1,1,1,1,1,1,6,1,1.0,572,2,1,2,1,1,1,5.0,5.0,2,1,1,0.0,1,1.0,2.0,1.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,1,1.0,2.0,2.0,2.0,0.0,2,1,1,2,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,1.0,400.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,1.0,1.0,3.0,575,U,1,1,11,206264,570,17,2,1,2
2,2,1,1,1,1,2,2,6,1,1.0,572,2,2,2,2,2,1,5.0,5.0,2,2,2,1.0,1,1.0,2.0,2.0,2.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,2,0.0,0.0,0.0,0.0,1.0,1,1,2,1,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,1.0,500.0,2.0,0.0,2.0,0.0,2.0,0.0,1.0,3.0,1.0,575,U,1,1,11,206264,570,17,2,1,3
3,2,1,1,1,1,1,2,7,1,1.0,572,1,1,1,2,2,1,2.0,2.0,2,2,2,1.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2,0.0,0.0,0.0,0.0,1.0,2,2,2,2,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,3.0,3.0,3.0,575,U,1,1,11,206264,570,17,2,1,4
4,2,1,1,1,1,1,1,6,1,1.0,572,1,1,2,1,1,1,6.0,6.0,1,1,1,0.0,1,1.0,2.0,1.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,2,0.0,0.0,0.0,0.0,2.0,1,1,2,1,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,1.0,460.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,3.0,2.0,575,U,1,1,11,206264,570,17,2,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58149,3,1,1,1,1,1,1,3,1,1.0,271,1,1,1,1,1,1,3.0,3.0,1,1,1,0.0,1,3.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,0.0,1,1.0,2.0,2.0,2.0,0.0,2,1,1,2,1,2,2.0,1.0,470.0,0.0,1.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,1.0,3.0,271,U,1,4,42,3201181,8162,340,32,1,1
58150,3,1,1,1,1,1,1,4,1,1.0,271,1,1,2,1,2,1,4.0,4.0,2,2,2,7.0,1,1.0,2.0,1.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,1,1.0,2.0,2.0,2.0,0.0,2,1,1,2,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,1.0,350.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,2.0,3.0,271,U,1,4,42,3201181,8162,340,32,1,2
58151,3,1,1,1,1,1,1,5,1,1.0,271,1,2,2,1,1,1,5.0,5.0,2,1,1,0.0,1,3.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,0.0,1,1.0,2.0,2.0,2.0,0.0,2,1,1,2,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,1.0,400.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,1.0,3.0,271,U,1,4,42,3201181,8162,340,32,1,3
58152,3,1,1,1,1,1,1,3,1,1.0,271,1,2,2,1,1,1,3.0,3.0,2,1,1,0.0,1,3.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,0.0,1,1.0,2.0,2.0,2.0,0.0,1,1,1,1,1,2,2.0,1.0,609.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,1.0,1.0,1.0,271,U,1,4,42,3201181,8162,340,32,1,4


In [11]:
hogares_agrupados["DOMINIO"] = hogares_agrupados.DOMINIO.apply(lambda x: x=="U").astype('int')

# ESTRATO BAJO

In [12]:
Xvivhog_bajo = hogares_agrupados[hogares_agrupados.ESTRATO==1]
Xvivhog_bajo

,P1_1,P1_2,P1_3,P1_4,P1_5_1,P1_5_2,P1_5_3,P2_1,P2_2,P2_3,FAC_VIV,P4_1_1,P4_1_2,P4_1_3,P4_1_4,P4_1_5,P4_1_6,P4_1_6_1,P4_1_6_2,P4_2_1,P4_2_2,P4_2_3,P4_3,P4_4,P4_5,P4_6_1,P4_6_2,P4_6_3,P4_6_4,P4_6_5,P4_6_6,P4_7_1,P4_7_2,P4_7_3,P4_7_4,P4_8,P5_1,P5_2_1,P5_2_2,P5_3_1,P5_3_2,P5_4,P5_5,P5_6_1,P5_6_2,P5_6_3,P5_6_4,P5_6_5,P5_7_1,P5_7_2,P5_8_2,P5_9_2,P5_10_2,P5_7_3,P5_8_3,P5_7_4,P5_8_4,P5_9_4,P5_7_5,P5_8_5,P5_7_6,P5_8_6,P5_7_7,P5_8_7,P5_7_8,P5_8_8,P5_11_1,P5_11_2,P5_11_3,FAC_HOG,DOMINIO,TLOC,ESTRATO,Grupo,UPM,UPM_DIS,EST_DIS,ENT,HOGAR,VIV_SEL
0,2,1,1,1,1,2,2,2,1,1.0,572,2,1,1,2,2,1,2.0,2.0,2,2,2,1.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2,0.0,0.0,0.0,0.0,1.0,2,2,2,2,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,3.0,3.0,3.0,575,1,1,1,11,206264,570,17,2,1,1
1,2,1,1,1,1,1,1,6,1,1.0,572,2,1,2,1,1,1,5.0,5.0,2,1,1,0.0,1,1.0,2.0,1.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,1,1.0,2.0,2.0,2.0,0.0,2,1,1,2,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,1.0,400.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,1.0,1.0,3.0,575,1,1,1,11,206264,570,17,2,1,2
2,2,1,1,1,1,2,2,6,1,1.0,572,2,2,2,2,2,1,5.0,5.0,2,2,2,1.0,1,1.0,2.0,2.0,2.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,2,0.0,0.0,0.0,0.0,1.0,1,1,2,1,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,1.0,500.0,2.0,0.0,2.0,0.0,2.0,0.0,1.0,3.0,1.0,575,1,1,1,11,206264,570,17,2,1,3
3,2,1,1,1,1,1,2,7,1,1.0,572,1,1,1,2,2,1,2.0,2.0,2,2,2,1.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2,0.0,0.0,0.0,0.0,1.0,2,2,2,2,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,3.0,3.0,3.0,575,1,1,1,11,206264,570,17,2,1,4
4,2,1,1,1,1,1,1,6,1,1.0,572,1,1,2,1,1,1,6.0,6.0,1,1,1,0.0,1,1.0,2.0,1.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,2,0.0,0.0,0.0,0.0,2.0,1,1,2,1,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,1.0,460.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,3.0,2.0,575,1,1,1,11,206264,570,17,2,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10944,2,1,1,1,1,1,2,2,1,1.0,197,2,1,2,2,2,1,1.0,1.0,2,1,2,0.0,1,1.0,2.0,2.0,1.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,2,0.0,0.0,0.0,0.0,1.0,1,1,2,1,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,1.0,9999.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,3.0,2.0,197,0,4,1,11,3260594,8230,336,32,1,14
10945,2,2,2,1,1,1,2,1,1,1.0,197,2,2,2,2,2,2,0.0,0.0,2,2,2,3.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,2,0.0,0.0,0.0,0.0,5.0,2,2,2,2,2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,197,0,4,1,11,3260594,8230,336,32,1,15
10946,2,1,1,1,1,2,2,4,1,1.0,197,1,2,2,1,2,2,0.0,0.0,2,2,2,1.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2,0.0,0.0,0.0,0.0,1.0,2,2,2,2,2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,197,0,4,1,11,3260594,8230,336,32,1,16
10947,2,2,1,1,1,2,1,2,1,1.0,197,2,2,2,2,2,2,0.0,0.0,2,2,2,3.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2,0.0,0.0,0.0,0.0,3.0,1,2,2,1,2,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,1.0,260.0,2.0,0.0,3.0,3.0,1.0,197,0,4,1,11,3260594,8230,336,32,1,18


In [13]:
cols_bajo = pd.read_csv("./columnas_clustering/cols_clus_bajo.csv").columns
modelo_bajo = pickle.load(open("./Modelos/agrupamiento_birch_bajo.pkl","rb"))

In [14]:
explainer_bajo = shap.Explainer(modelo_bajo.predict, Xvivhog_bajo[cols_bajo])

In [39]:
hogares_extended = []
for i in Xvivhog_bajo.index:
    reps = Xvivhog_bajo.loc[i,"FAC_HOG"]
    df_temp = Xvivhog_bajo.loc[[i]].copy()
    df_temp["FAC_HOG FAC_VIV".split()] = 1
    tot_trans = np.tile(df_temp, (reps,1))
    hogares_extended.append(pd.DataFrame(tot_trans))
    del df_temp

In [41]:
Xvivhog_bajoext = pd.concat(hogares_extended, ignore_index=True)
Xvivhog_bajoext.columns = Xvivhog_bajo.columns
del hogares_extended
Xvivhog_bajoext


,P1_1,P1_2,P1_3,P1_4,P1_5_1,P1_5_2,P1_5_3,P2_1,P2_2,P2_3,FAC_VIV,P4_1_1,P4_1_2,P4_1_3,P4_1_4,P4_1_5,P4_1_6,P4_1_6_1,P4_1_6_2,P4_2_1,P4_2_2,P4_2_3,P4_3,P4_4,P4_5,P4_6_1,P4_6_2,P4_6_3,P4_6_4,P4_6_5,P4_6_6,P4_7_1,P4_7_2,P4_7_3,P4_7_4,P4_8,P5_1,P5_2_1,P5_2_2,P5_3_1,P5_3_2,P5_4,P5_5,P5_6_1,P5_6_2,P5_6_3,P5_6_4,P5_6_5,P5_7_1,P5_7_2,P5_8_2,P5_9_2,P5_10_2,P5_7_3,P5_8_3,P5_7_4,P5_8_4,P5_9_4,P5_7_5,P5_8_5,P5_7_6,P5_8_6,P5_7_7,P5_8_7,P5_7_8,P5_8_8,P5_11_1,P5_11_2,P5_11_3,FAC_HOG,DOMINIO,TLOC,ESTRATO,Grupo,UPM,UPM_DIS,EST_DIS,ENT,HOGAR,VIV_SEL
0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,3.0,3.0,3.0,1.0,1.0,1.0,1.0,11.0,206264.0,570.0,17.0,2.0,1.0,1.0
1,2.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,3.0,3.0,3.0,1.0,1.0,1.0,1.0,11.0,206264.0,570.0,17.0,2.0,1.0,1.0
2,2.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,3.0,3.0,3.0,1.0,1.0,1.0,1.0,11.0,206264.0,570.0,17.0,2.0,1.0,1.0
3,2.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,3.0,3.0,3.0,1.0,1.0,1.0,1.0,11.0,206264.0,570.0,17.0,2.0,1.0,1.0
4,2.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,3.0,3.0,3.0,1.0,1.0,1.0,1.0,11.0,206264.0,570.0,17.0,2.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6984834,2.0,1.0,2.0,1.0,1.0,1.0,2.0,4.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,0.0,0.0,2.0,2.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4.0,1.0,11.0,3260594.0,8230.0,336.0,32.0,1.0,20.0
6984835,2.0,1.0,2.0,1.0,1.0,1.0,2.0,4.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,0.0,0.0,2.0,2.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4.0,1.0,11.0,3260594.0,8230.0,336.0,32.0,1.0,20.0
6984836,2.0,1.0,2.0,1.0,1.0,1.0,2.0,4.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,0.0,0.0,2.0,2.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4.0,1.0,11.0,3260594.0,8230.0,336.0,32.0,1.0,20.0
6984837,2.0,1.0,2.0,1.0,1.0,1.0,2.0,4.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,0.0,0.0,2.0,2.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4.0,1.0,11.0,3260594.0,8230.0,336.0,32.0,1.0,20.0


In [ ]:
%time
patrones_bajos = pyfpgrowth.find_frequent_patterns(Xvivhog_bajoext[cols_bajo],2)
patrones_bajos

# ESTRATO MEDIO BAJO

In [36]:
Xvivhog_mbajo = hogares_agrupados[hogares_agrupados.ESTRATO==2]
Xvivhog_mbajo

,P1_1,P1_2,P1_3,P1_4,P1_5_1,P1_5_2,P1_5_3,P2_1,P2_2,P2_3,FAC_VIV,P4_1_1,P4_1_2,P4_1_3,P4_1_4,P4_1_5,P4_1_6,P4_1_6_1,P4_1_6_2,P4_2_1,P4_2_2,P4_2_3,P4_3,P4_4,P4_5,P4_6_1,P4_6_2,P4_6_3,P4_6_4,P4_6_5,P4_6_6,P4_7_1,P4_7_2,P4_7_3,P4_7_4,P4_8,P5_1,P5_2_1,P5_2_2,P5_3_1,P5_3_2,P5_4,P5_5,P5_6_1,P5_6_2,P5_6_3,P5_6_4,P5_6_5,P5_7_1,P5_7_2,P5_8_2,P5_9_2,P5_10_2,P5_7_3,P5_8_3,P5_7_4,P5_8_4,P5_9_4,P5_7_5,P5_8_5,P5_7_6,P5_8_6,P5_7_7,P5_8_7,P5_7_8,P5_8_8,P5_11_1,P5_11_2,P5_11_3,FAC_HOG,DOMINIO,TLOC,ESTRATO,Grupo,UPM,UPM_DIS,EST_DIS,ENT,HOGAR,VIV_SEL
10949,3,1,1,1,1,2,2,1,1,1.0,555,2,1,1,2,2,1,1.0,0.0,2,2,2,2.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2,0.0,0.0,0.0,0.0,1.0,2,2,2,2,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,3.0,3.0,3.0,557,1,1,2,20,206599,585,17,2,1,1
10950,3,1,1,1,1,1,1,7,1,1.0,555,1,1,2,1,2,1,7.0,7.0,1,1,1,0.0,1,3.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,0.0,1,1.0,2.0,2.0,2.0,0.0,1,1,1,1,1,2,2.0,1.0,560.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,1.0,1.0,1.0,557,1,1,2,20,206599,585,17,2,1,2
10951,2,1,1,1,1,1,1,3,1,1.0,555,2,1,2,2,2,1,2.0,2.0,2,2,2,1.0,1,3.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,0.0,1,1.0,2.0,2.0,2.0,0.0,2,1,1,2,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,1.0,400.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,8.0,3.0,557,1,1,2,20,206599,585,17,2,1,3
10952,2,1,1,1,1,1,1,5,1,1.0,555,1,2,2,1,2,1,3.0,2.0,1,1,1,0.0,1,3.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,0.0,1,1.0,2.0,2.0,2.0,0.0,1,1,1,1,1,2,2.0,1.0,540.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,1.0,1.0,1.0,557,1,1,2,20,206599,585,17,2,1,4
10953,2,1,1,1,1,1,2,5,1,1.0,555,2,2,2,1,2,1,1.0,1.0,2,2,2,2.0,1,3.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,0.0,1,1.0,2.0,1.0,2.0,0.0,2,1,1,2,1,2,2.0,1.0,200.0,0.0,1.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,1.0,1.0,3.0,557,1,1,2,20,206599,585,17,2,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41147,2,1,1,1,1,1,1,7,1,1.0,199,1,1,2,1,2,1,1.0,0.0,2,2,2,1.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2,0.0,0.0,0.0,0.0,1.0,1,2,2,1,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,1.0,300.0,2.0,0.0,3.0,3.0,1.0,199,0,4,2,20,3260632,8231,337,32,1,5
41148,2,1,1,1,1,1,1,6,2,2.0,199,2,2,2,1,2,1,1.0,1.0,2,2,1,0.0,1,1.0,2.0,2.0,1.0,1.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,2,0.0,0.0,0.0,0.0,1.0,1,1,2,1,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,1.0,20.0,1.0,200.0,1.0,3.0,1.0,199,0,4,2,20,3260632,8231,337,32,1,6
41149,2,1,1,1,1,1,1,6,2,2.0,199,2,2,1,1,2,1,2.0,2.0,2,2,2,2.0,1,1.0,2.0,1.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,1,1.0,1.0,2.0,2.0,0.0,2,1,1,2,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,1.0,169.0,2.0,0.0,1.0,300.0,1.0,1.0,3.0,199,0,4,2,20,3260632,8231,337,32,2,6
41150,3,1,1,1,1,1,1,6,1,1.0,199,1,2,2,1,2,1,1.0,1.0,2,2,2,1.0,1,1.0,2.0,2.0,1.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,2,0.0,0.0,0.0,0.0,1.0,2,1,2,2,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,1.0,0.0,1.0,3.0,3.0,199,0,4,2,20,3260632,8231,337,32,1,7


In [37]:
cols_medio_bajo = pd.read_csv("./columnas_clustering/cols_clus_medio_bajo.csv").columns
modelo_medio_bajo = pickle.load(open("./Modelos/agrupamiento_birch_medio_bajo.pkl","rb"))

In [32]:
explainer_mbajo = shap.Explainer(modelo_medio_bajo.predict, Xvivhog_mbajo[cols_medio_bajo])

In [39]:
shaps_mbajo = explainer_mbajo.shap_values(Xvivhog_mbajo[cols_medio_bajo])
shaps_mbajo

PermutationExplainer explainer:  10%|█         | 3112/30203 [1:33:45<13:38:05,  1.81s/it]


KeyboardInterrupt: 